In [3]:
import os 

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# 오디오 파일 경로
snu_data_sample_path = "/workspace/kor_med_stt_data/snu_data/강지예_201812001_행동치료.wav"
kru_data_sample_path = "/workspace/kru_data/원천데이터/의사/HB_0276/HB_0276-1004-01-01-M-04-A.wav"

# Whisper-base 모델과 프로세서 로드 (한국어)
processor = WhisperProcessor.from_pretrained("openai/whisper-base", language="ko", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-base").to("cuda" if torch.cuda.is_available() else "cpu")

# 오디오 로드
audio, sr = librosa.load(
    snu_data_sample_path
    # kru_data_sample_path
    , sr=16000)

# 입력 특성 추출
input_features = processor.feature_extractor(
    audio, sampling_rate=16000, return_tensors="pt"
).input_features.to(model.device)

# 예측
with torch.no_grad():
    predicted_ids = model.generate(input_features)
    transcription = processor.tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)[0]

print("Whisper-base 예측 결과:", transcription)

Whisper-base 예측 결과:  근데 행동치료 감각통업치고 가는 게 발달이 돼서 이런 치료를 하면 굉장히 훨씬 나아질 수가 있거든요.


In [4]:
import os
import numpy as np
import librosa
import torch
from transformers import AutoProcessor, WhisperForConditionalGeneration

# 모델 체크포인트 디렉토리
custom_ckpt_dir = "/workspace/results/whisper_train/whisper-base/checkpoint-5090605"

# WhisperInference 클래스 정의 참고(위의 whisper_tester.py 기반)
class WhisperInference:
    def __init__(
        self,
        model_dir: str,
        sampling_rate: int = 16000,
        device: str = "cuda",
        init_batch_size: int = 64
    ):
        self.model_dir = model_dir
        self.sampling_rate = sampling_rate
        self.device = device
        self.init_batch_size = init_batch_size

        model_name = os.path.basename(os.path.dirname(self.model_dir))
        hf_model_id = f"openai/{model_name}"

        # 확실하게 processor/model이 문제가 없는지 로그 찍기
        print(f"Processor 로드: {hf_model_id}")
        self.processor = AutoProcessor.from_pretrained(
            hf_model_id, language="ko", task="transcribe"
        )
        print(f"Model 로드: {model_dir}")
        self.model = WhisperForConditionalGeneration.from_pretrained(model_dir).to(device)

    def transcribe(self, audio_path):
        print(f"오디오 입력: {audio_path}")
        # 오디오 로드
        try:
            audio, _ = librosa.load(audio_path, sr=self.sampling_rate)
            print(f"오디오 shape: {audio.shape}, dtype: {audio.dtype}")
        except Exception as e:
            print("오디오 로드 오류:", e)
            audio = np.zeros(self.sampling_rate, dtype=np.float32)
            print("0으로 대체")

        feats = self.processor.feature_extractor(
            audio, sampling_rate=self.sampling_rate, return_tensors="pt"
        ).input_features.to(self.device)
        print(f"input_features shape: {feats.shape}")

        with torch.no_grad():
            pred_ids = self.model.generate(feats)
        text = self.processor.batch_decode(pred_ids, skip_special_tokens=True)[0]
        print("디코딩 결과:", text)
        return text

whisper_infer = WhisperInference(
    model_dir=custom_ckpt_dir,
    device="cuda" if torch.cuda.is_available() else "cpu"
)

custom_ckpt_transcription = whisper_infer.transcribe(
    snu_data_sample_path
    # kru_data_sample_path
    )

print("Custom checkpoint 예측 결과:", custom_ckpt_transcription)

if not custom_ckpt_transcription or custom_ckpt_transcription.strip() == "":
    print("⚠️ 결과가 비어 있습니다. 모델이나 체크포인트, 입력 오디오 경로 등을 다시 확인하십시오.")
else:
    print("✅ 결과가 정상적으로 출력되었습니다.")


Processor 로드: openai/whisper-base
Model 로드: /workspace/results/whisper_train/whisper-base/checkpoint-5090605
오디오 입력: /workspace/kor_med_stt_data/snu_data/강지예_201812001_행동치료.wav
오디오 shape: (125675,), dtype: float32
input_features shape: torch.Size([1, 80, 3000])
디코딩 결과: 생활이��니다. 이런 치료를 서명할 수 있는 때까지 나아질 수 있거든요.
Custom checkpoint 예측 결과: 생활이��니다. 이런 치료를 서명할 수 있는 때까지 나아질 수 있거든요.
✅ 결과가 정상적으로 출력되었습니다.


'\n'